In [196]:
import random
import os
from pprint import pprint

import requests
from lxml.html import fromstring

import pandas as pd

In [197]:
def random_user_agent(file):
    if os.path.exists(file):
        with open(file, 'r') as f:
            lines = f.readlines()

            return str(random.choice(lines)).replace("\n", "")

In [198]:
url_list = [
    'https://www.laliga.es/laliga-santander/REPLACE_TEAM/plantilla/clasicas',
    'https://www.laliga.es/laliga-santander/REPLACE_TEAM/plantilla/defensivas',
    'https://www.laliga.es/laliga-santander/REPLACE_TEAM/plantilla/disciplina',
    'https://www.laliga.es/laliga-santander/REPLACE_TEAM/plantilla/ofensivas',
    'https://www.laliga.es/laliga-santander/REPLACE_TEAM/plantilla/eficiencia',
]

In [199]:
def retrieve_team_list():
    head = {
        "User-Agent": random_user_agent('resources/user_agent_list.txt'),
        "X-Requested-With": "XMLHttpRequest",
        "Accept": "text/html,application/xml",
        "Accept-Encoding": "gzip, deflate, br",
        "Connection": "keep-alive",
    }

    url = "https://www.laliga.es/laliga-santander"

    req = requests.get(url, headers=head)
    
    if req.status_code != 200:
        raise ConnectionError("connection error " + str(req.status_code) + ", try again later.")

    root = fromstring(req.text)
    path = root.xpath(".//div[@id='equipos']/div[contains(@class, 'laliga-santander')]")
    
    strip_from_url = 'https://www.laliga.es/laliga-santander/'
    
    teams = list()
    
    for element in path:
        for value in element:
            teams.append(value.get('href').replace(strip_from_url, ''))
            
    return teams

In [200]:
def retrieve_team_dict():
    head = {
        "User-Agent": random_user_agent('resources/user_agent_list.txt'),
        "X-Requested-With": "XMLHttpRequest",
        "Accept": "text/html,application/xml",
        "Accept-Encoding": "gzip, deflate, br",
        "Connection": "keep-alive",
    }

    url = "https://www.laliga.es/laliga-santander"

    req = requests.get(url, headers=head)
    
    if req.status_code != 200:
        raise ConnectionError("connection error " + str(req.status_code) + ", try again later.")

    root = fromstring(req.text)
    path = root.xpath(".//div[@id='equipos']/div[contains(@class, 'laliga-santander')]")
    
    strip_from_url = 'https://www.laliga.es/laliga-santander/'
    
    teams = list()
    
    for element in path:
        for value in element:
            obj = {
                'name': value.text_content(),
                'tag': value.get('href').replace(strip_from_url, ''),
            }
            
            teams.append(obj)
            
    return teams

In [201]:
def retrieve_player_stats(team):
    result = list()
    
    for url in url_list:
        url = url.replace('REPLACE_TEAM', team['tag'])
        
        head = {
            "User-Agent": random_user_agent('resources/user_agent_list.txt'),
            "X-Requested-With": "XMLHttpRequest",
            "Accept": "text/html,application/xml",
            "Accept-Encoding": "gzip, deflate, br",
            "Connection": "keep-alive",
        }

        req = requests.get(url, headers=head)

        if req.status_code != 200:
            raise ConnectionError("connection error " + str(req.status_code) + ", try again later.")

        root = fromstring(req.text)
        path = root.xpath(".//div[contains(@class, 'rotar-tabla')]/table[@class='datatable']/thead/tr/th")

        index = list()

        for element in path:
            if 'title' not in element.attrib:
                index.append(element.text)
            else:
                index.append(element.get('title'))

        df = pd.DataFrame(columns=index)

        path = root.xpath(".//div[contains(@class, 'rotar-tabla')]/table[@class='datatable']/tbody/tr")

        row = list()
        
        for element in path:
            for value in element:
                row.append(value.text_content())

            df.loc[len(df)] = row
            row = list()
        
        df.insert(loc=0, column='Equipo', value=team['name'])
        df = df.drop(columns=['Foto'])
        
        result.append(df)
        
    return result

In [202]:
def concatenate_dataframes(df_list):
    result = pd.concat([df for df in df_list], axis=1)
    result = result.loc[:,~result.columns.duplicated()]
    
    return result

In [203]:
def __launch__():
    team_dict = retrieve_team_dict()
    players = pd.DataFrame()
    
    for team in team_dict:
        df_list = retrieve_player_stats(team)
        
        players = players.append(concatenate_dataframes(df_list), ignore_index=True, sort=False)
        
    # LaLiga Error -> https://www.laliga.es/laliga-santander/barcelona/plantilla/eficiencia
    players = players.drop(columns=['Goles marcados por tiros realizados desde fuera del área',
                                    'Goles marcados por tiros realizados desde dentro del área',
                                    'Goles marcados por tiros realizados con el pie izquierdo',
                                    'Goles marcados por tiros realizados con el pie derecho',
                                    'Goles marcados por tiros realizados de cabeza',
                                    'Goles marcados por tiros realizados de jugada a balón parado'])
    
    players.to_csv('../dataset/laliga_player_stats.csv', encoding='utf-8', index=False)
    
    return players

In [204]:
players = __launch__()

In [205]:
players.head()

,Equipo,Posicion,Dorsal,Nombre,Minutos jugados,Partidos jugados,Porcentaje de Partidos jugados,Partidos jugados enteros,Porcentaje de Partidos jugados enteros,Partidos jugados como titular,...,Corneres lanzados,Entradas realizadas,Duelos realizados,Duelos cuerpo a cuerpo realizados,Duelos aéreos realizados,Pases,Pases cortos,Pases largos,Pases al hueco,Goles marcados por tiros realizados
0,Athletic Club,Portero,,Hodei Oleaga,0,0,"0,0%",0,"0,0%",0,...,0,0,0,0,0,0,0,0,0,0
1,Athletic Club,Portero,1,A. Remiro,0,0,"0,0%",0,"0,0%",0,...,0,0,0,0,0,0,0,0,0,0
2,Athletic Club,Portero,13,Herrerín,2.790,31,"82,0%",31,"82,0%",31,...,0,0,25,6,19,887,128,759,1,0
3,Athletic Club,Portero,25,Unai Simón,630,7,"18,0%",7,"18,0%",7,...,0,0,3,2,1,155,49,106,0,0
4,Athletic Club,Defensa,3,Núñez,1.063,12,"32,0%",11,"29,0%",11,...,0,15,107,38,69,536,457,78,1,0
